<a href="https://colab.research.google.com/github/JCaballerot/Deep_learning_program/blob/main/Topicos_avanzados/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><font size="5">CIFAR10</font></h1>


<center>
  <img src="https://symjax.readthedocs.io/en/latest/_images/sphx_glr_plot_cifar10_001.svg" width="800" height="300">
</center>


Este laboratorio se enfoca en la construcción, entrenamiento y optimización de un modelo de red neuronal convolucional (CNN) para clasificar imágenes del conjunto de datos CIFAR10, que consiste en 60,000 imágenes a color de 32x32 distribuidas en 10 clases diferentes.

### 1. Instalación de dependencias


Primero, instalamos scikeras, que es una interfaz de Scikit-learn para Keras, permitiéndonos usar herramientas de Scikit-learn como GridSearchCV para optimizar nuestros modelos de Keras.

In [1]:
%%capture
!pip3 install scikeras

---

### 2. Importación de librerías necesarias

Aquí importamos todas las librerías necesarias para nuestro laboratorio, incluyendo TensorFlow, Keras para la construcción del modelo, y Scikit-learn para la optimización de hiperparámetros.

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier


---

### 3. Definición del modelo


En esta sección, definimos la arquitectura de nuestra red neuronal convolucional (CNN), especificando capas convolucionales, capas de pooling, y capas densas para la clasificación. También configuramos dinámicamente el optimizador, la tasa de aprendizaje y la función de pérdida.

In [3]:

def create_model(learning_rate=0.01, optimizer='adam', loss='sparse_categorical_crossentropy'):

    model = Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),  # Agregar Dropout después de la capa de MaxPooling
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),  # Agregar Dropout antes de la capa final
        Dense(10, activation='softmax')
    ])

    optimizer_choices = {
        'adam': Adam(learning_rate=learning_rate),
        'sgd': keras.optimizers.SGD(learning_rate=learning_rate),
        'rmsprop': keras.optimizers.RMSprop(learning_rate=learning_rate)
    }

    model.compile(optimizer=optimizer_choices[optimizer],
                  loss=loss,
                  metrics=['accuracy'])
    return model

### 4. Preprocesamiento de datos


Cargamos y preprocesamos el conjunto de datos CIFAR10, normalizando las imágenes y convirtiendo las etiquetas a formato one-hot para su uso con la función de pérdida categorical_crossentropy.

In [4]:
from tensorflow.keras.utils import to_categorical

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [6]:
# Muestrear 2000 datos aleatoriamente de cada conjunto (entrenamiento y prueba)
indices_train = np.random.choice(range(x_train.shape[0]), size = 1000, replace=False)
indices_test = np.random.choice(range(x_test.shape[0]), size = 1000, replace=False)

x_train_sampled = x_train[indices_train]
y_train_sampled = y_train[indices_train]

x_test_sampled = x_test[indices_test]
y_test_sampled = y_test[indices_test]

In [7]:

# Normalizar los datos
x_train_sampled, x_test_sampled = x_train_sampled / 255.0, x_test_sampled / 255.0

# Para usar 'categorical_crossentropy', necesitamos etiquetas en formato one-hot
y_train_one_hot = to_categorical(y_train_sampled, 10)
y_test_one_hot = to_categorical(y_test_sampled, 10)

### 5. Configuración de Gridsearch


Establecemos una grilla de hiperparámetros para explorar diferentes configuraciones de tasa de aprendizaje, tamaño de lote, optimizadores y funciones de pérdida, y preparamos nuestro modelo para la optimización.

In [8]:
param_grid = {
    'model__learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [64, 128, 256],
    'model__optimizer': ['adam', 'sgd', 'rmsprop'],  # optimizador
    'model__loss': ['sparse_categorical_crossentropy']  # función de pérdida
}

### 6. Ejecución de la búsqueda de hiperparámetros y resultados


Finalmente, ejecutamos la búsqueda de hiperparámetros utilizando GridSearchCV, entrenando el modelo con diferentes configuraciones y evaluando su rendimiento. Los mejores parámetros se muestran al final, junto con el rendimiento de todas las configuraciones probadas.

In [11]:
model = KerasClassifier(build_fn = create_model, epochs = 10, verbose=1)  # Ajusta epochs según necesidad

grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = 1, cv = 3, verbose = 2)


In [ ]:
# Usa y_train_one_hot si eliges 'categorical_crossentropy', y_train para 'sparse_categorical_crossentropy'
grid_result = grid.fit(x_train_sampled, y_train_sampled)


In [ ]:

# Mostrar los resultados
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds  = grid_result.cv_results_['std_test_score']

params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) con: %r" % (mean, stdev, param))


In [15]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Asume que grid_result es el resultado de tu GridSearchCV
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

results_df = pd.DataFrame(params)
results_df['Mean Accuracy'] = means
results_df['Std Deviation'] = stds

# Opcionalmente, podrías querer añadir una columna ID para identificar cada configuración
results_df['Config ID'] = results_df.index

results_df

In [ ]:
# Selecciona el hiperparámetro para visualizar
hyperparam = 'model__optimizer'  # Este es el hiperparámetro de interés

# Preparando los datos para el gráfico
# Agrupando por el hiperparámetro seleccionado y calculando la media y desviación estándar de la precisión para cada valor
grouped = results_df.groupby(hyperparam)['Mean Accuracy'].agg(['mean', 'std']).reset_index()

# Graficar los resultados
plt.figure(figsize=(10, 6))
plt.bar(grouped[hyperparam], grouped['mean'], yerr=grouped['std'], color='skyblue', capsize=5)
plt.title(f'Impacto de {hyperparam.capitalize()} en la Precisión Media')
plt.xlabel(hyperparam.capitalize())
plt.ylabel('Precisión Media')
plt.xticks(rotation=45)  # Rota las etiquetas si son largas o numerosas
plt.tight_layout()
plt.show()

### 6. Conclusión

En este laboratorio, aplicamos técnicas de optimización de hiperparámetros en un modelo de red neuronal convolucional (CNN) para clasificar imágenes del conjunto de datos CIFAR-10. A través del uso de GridSearchCV con KerasClassifier, exploramos combinaciones de hiperparámetros críticos, incluyendo la tasa de aprendizaje, el tamaño de lote, el optimizador y la función de pérdida, buscando la configuración óptima que maximizara la precisión de nuestro modelo.

Los resultados obtenidos demuestran que la optimización de hiperparámetros juega un papel crucial en el desempeño de los modelos de aprendizaje profundo. Identificamos la combinación de hiperparámetros que proporcionó la mayor precisión en la clasificación de las imágenes CIFAR-10, lo cual subraya la importancia de un proceso de selección y ajuste meticuloso.

Este laboratorio también destaca la potencia y flexibilidad de scikeras y GridSearchCV para integrar modelos de Keras en flujos de trabajo de aprendizaje automático con scikit-learn, facilitando así la experimentación y la optimización de modelos de aprendizaje profundo.

---
## Gracias por completar este laboratorio!